In [1]:
import os
import re
import json

from multiprocessing import cpu_count
num_proc = cpu_count()

import yaml

from dataprep.stix.StixConfig import StixToPydanticMap, STIX
from pydantic import BaseModel, ValidationError


from evaluation.stix_evaluator import STIXEvaluator

from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template
import torch

from trl import GRPOConfig, GRPOTrainer

from data_processor import SplittedJsonIoDataset
from customs import customize_tokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-03 12:27:57 [__init__.py:244] Automatically detected platform cuda.


In [2]:
# Clear GPU cache
torch.cuda.empty_cache()

In [3]:
with open("grpo_config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

In [5]:
# # Check if model is a peft model
# import peft.helpers

# def load_model_and_tokenizer(model_name_or_path, config, **kwargs):
#     if peft.helpers.check_if_peft_model(model_name_or_path):
#         model, tokenizer = FastLanguageModel.from_pretrained(
#                 model_name=model_name_or_path,
#                 **kwargs
#             )
#         if config["merge_peft_model"]:
#             model.merge_and_unload()
#     else:
#         model, tokenizer = FastLanguageModel.from_pretrained(
#             **config["model_loading_args"]
#         )
#     return model, tokenizer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-0528-Qwen3-8B", #"/mnt/data/training-outputs/first-run/outputs/checkpoint-194",
    fast_inference = True,
    max_seq_length = 65536
)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


==((====))==  Unsloth 2025.6.8: Fast Qwen3 patching. Transformers: 4.53.0. vLLM: 0.9.1.
   \\   /|    NVIDIA H100 PCIe. Num GPUs = 1. Max memory: 79.19 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: vLLM loading unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit with actual GPU utilization = 49.66%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.19 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 65536. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up to 32.64 GB. Also swap space = 6 GB.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


...

INFO 07-03 13:06:17 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 07-03 13:06:17 [gpu_model_runner.py:1624] Model loading took 7.3485 GiB and 15.758609 seconds
INFO 07-03 13:06:28 [backends.py:462] Using cache directory: /home/deleftheriou/.cache/vllm/torch_compile_cache/56d0d6642f/rank_0_0 for vLLM's torch.compile
INFO 07-03 13:06:28 [backends.py:472] Dynamo bytecode transform time: 10.42 s


Inductor Compilation: 100%|██████████| 6/6 [00:01<00:00,  4.75it/s, triton_poi_fused_add_mul_sub_5]                              

INFO 07-03 13:06:32 [backends.py:161] Cache the graph of shape None for later use



Inductor Compilation: 100%|██████████| 5/5 [00:00<00:00, 16.64it/s, triton_red_fused__to_copy_add_mean_mul_pow_rsqrt_4] 


INFO 07-03 13:07:08 [backends.py:173] Compiling a graph for general shape takes 38.33 s
INFO 07-03 13:09:08 [monitor.py:34] torch.compile takes 48.75 s in total
INFO 07-03 13:09:11 [gpu_worker.py:227] Available KV cache memory: 24.07 GiB
INFO 07-03 13:09:11 [kv_cache_utils.py:715] GPU KV cache size: 175,264 tokens
INFO 07-03 13:09:11 [kv_cache_utils.py:719] Maximum concurrency for 65,536 tokens per request: 2.67x
INFO 07-03 13:10:16 [gpu_model_runner.py:2048] Graph capturing finished in 65 secs, took 1.71 GiB
INFO 07-03 13:10:17 [core.py:171] init engine (profile, create kv cache, warmup model) took 239.11 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']


In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 8,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth 2025.6.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
#model, tokenizer = customize_tokenizer(model, tokenizer, config)
# no need for deepseek
# config["chat_template"] = "deepseek"
# tokenizer = get_chat_template(tokenizer, config["chat_template"])

In [9]:
# Create dataset for training
dataset = SplittedJsonIoDataset(tokenizer, config).grpo_create()

Map:   0%|          | 0/1564 [00:00<?, ? examples/s]

Map:   0%|          | 0/207 [00:00<?, ? examples/s]

In [18]:
def deserialize_answer(answer: str) -> dict:
    return json.loads(answer)

def deserialize_response_for_evaluation(answer: str) -> dict:
    if is_stix_bundle(answer):
        return json.loads(answer)
    else:
        return {"id":"", "type":"bundle", "objects":[]}

def extract_xml_answer(response: str) -> str:
    answer = response.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def is_stix_bundle(text: str) -> bool:
    try:
        bundle = json.loads(text)
        pydantic_stix_bundle = STIX(**bundle)
        return True
    except:
        return False

def count_valid_stix_objects(text: str) -> bool:
    smap = StixToPydanticMap()
    cnt = 0.0
    if is_stix_bundle(text):
        bundle = json.loads(text)
        for obj in bundle["objects"]:
            try:
                smap(obj)
                cnt += 1
            except ValidationError:
                pass
        return cnt / len(bundle["objects"])
    else:
      return cnt

In [19]:
def format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def stix_validity_reward_func(completions, answers, **kwargs) -> list[float]:
    """Reward function that checks if the completion can is a stix bundle."""
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if is_stix_bundle(r) else 0.0 for r in extracted_responses]

def stix_objects_validity_reward_func(completions, answers, **kwargs) -> list[float]:
    """Reward function that checks if the completion has valid stix objects."""
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 * count_valid_stix_objects(r) for r in extracted_responses]

def accuracy_reward_func(completions, answers, **kwargs) -> list[float]:
    evaluator = STIXEvaluator()
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    desirialized_responses = [deserialize_response_for_evaluation(r) for r in extracted_responses]
    desirialized_answers = [deserialize_answer(a) for a in answers]
    return [evaluator.evaluate_single(r, a)[2] for r, a in zip(desirialized_responses, desirialized_answers)]

In [20]:
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 16, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = config["model_loading_args"]["max_seq_length"],
    max_completion_length = config["model_loading_args"]["max_seq_length"],
    num_train_epochs = 1, # Set to 1 for a full training run
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "tensorboard", # Can use Weights & Biases
    output_dir = "grpo_outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 4 to the `num_generations` of 8


In [21]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        format_reward_func,
        stix_validity_reward_func,
        stix_objects_validity_reward_func,
        accuracy_reward_func
    ],
    args = training_args,
    train_dataset = dataset["train"],
    eval_dataset = dataset["eval"]
)

In [ ]:
from unsloth import unsloth_train
# Start training
trainer_stats = unsloth_train(trainer)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,564 | Num Epochs = 1 | Total steps = 98
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 16 x 1) = 128
 "-____-"     Trainable parameters = 21,823,488/8,000,000,000 (0.27% trained)


ERROR 07-03 13:17:23 [dump_input.py:69] Dumping input data
ERROR 07-03 13:17:23 [dump_input.py:71] V1 LLM engine (v0.9.1) with config: model='unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit', speculative_config=None, tokenizer='unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=65536, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None), seed=0, 

KeyboardInterrupt: 

: 